In [254]:
import re


match_num = re.compile(r'-?\d+').match


class Cruncher:
    def __init__(self, instructions, debug=False, letter=False, killstep=0):
        self.killstep = killstep
        self.debug = debug
        self.letter = letter
        self.instructions = [i.split() for i in instructions.splitlines() if i and not i.startswith('#')]
        self.numq = []
        self.reset()

    def reset(self):
        self.zwrites = 0
        self.wrefs = 0
        self.w = 0
        self.x = 0
        self.y = 0
        self.z = 0

    def inp (self, destreg):
        if not self.numq:
            print ("dead early")
        number = self.numq.pop()
        setattr(self, destreg, number)
        return f'inp {destreg} = {number}'

    def convert(self,other):
        if match_num(other):
            return int(other)
        return getattr(self, other)

    @staticmethod
    def get_other_msg(other, otherval):
        if other.isdigit():
            return f"{other}" 
        return f"{other} ({otherval})"

        
    def mod(self, destreg, other):
        otherval = self.convert(other)
        destval = getattr(self, destreg)
        result = destval % otherval
        setattr(self, destreg, result)
        other_msg = self.get_other_msg(other, otherval)
        return f'mod {destreg} ({destval}) % {other_msg} = {result}'

    def mul(self, destreg, other):
        otherval = self.convert(other)
        destval = getattr(self, destreg)
        result = destval * otherval
        setattr(self, destreg, result)
        other_msg = self.get_other_msg(other, otherval)
        return f'mul {destreg} ({destval}) * {other_msg} = {result}'

    def eql(self, destreg, other):
        otherval = self.convert(other)
        destval = getattr(self, destreg)
        result = int(destval == otherval)
        setattr(self, destreg, result)
        other_msg = self.get_other_msg(other, otherval)
        return f'eql {destreg} ({destval}) == {other_msg} = {result}'

    def div(self, destreg, other):
        otherval = self.convert(other)
        destval = getattr(self, destreg)
        result = destval // otherval
        setattr(self, destreg, result)
        other_msg = self.get_other_msg(other, otherval)
        return f'div {destreg} ({destval}) // {other_msg} = {result}'

    def add(self, destreg, other):
        otherval = self.convert(other)
        destval = getattr(self, destreg)
        result = destval + otherval
        setattr(self, destreg, result)
        other_msg = self.get_other_msg(other, otherval)
        return f'add {destreg} ({destval}) + {other_msg} = {result}'


    def crunch(self, numlist):
        self.debug and print (f'Crunching: {numlist}')
        self.numq = numlist
        self.reset()
        inp = 1
        for i, inst in enumerate(self.instructions, start=1):
            func = getattr(self, inst[0])
            msg = func(*inst[1:])
            if inst[0] == "inp":
                self.debug and print (f"Input {inp}")
                inp += 1
            if inst[1] == "w" or (len(inst) == 3 and inst[2] == "w"):
                self.wrefs += 1
                self.debug and print (f"{i:>3}:  {msg:<30} <-- !    ", end="")
            elif inst[0] == "mul" and inst[2] == "0":
                self.debug and print (f"---\n{i:>3}:  {msg:<40}", end="")
            else:
                self.debug and print (f"{i:>3}:  {msg:<40}", end="")
            self.debug and self.print_regs()

            if inst[1] == "z":
                self.zwrites += 1
                self.debug and print ("")
            if i == self.killstep:
                self.debug and print (f"Broke at step {i}")
                break
        else:
            self.debug and print("normal ending")
        self.debug and self.print_regs()


    def print_regs(self):
        w=self.w
        x=self.x
        y=self.y
        z=self.z
        print (f"{w=} {x=} {y=} {z=}")



def get_list(num):
    numstr = str(num)
    if "0" in numstr:
        z = 1
        return
    if len(numstr) != 14:
        return []
    if num < 11_111_111_111_111:
        return []
    numlist = [int(i) for i in str(num)]
    numlist.reverse()
    return numlist



num_list = get_list(12444444444443)


In [14]:

sample = """inp x
mul x -1"""

monad = Cruncher(sample, debug=True)
monad.crunch([1,2])


Crunching: [1, 2]
inp x = number
mul x (2) * -1 (-1) = -2
normal ending
self.w=0 self.x=-2 self.y=0 self.z=0


In [15]:

test_scale = """inp z
inp x
mul z 3
eql z x"""


monad = Cruncher(test_scale, debug=True)
monad.crunch([6,2])


Crunching: [6, 2]
inp z = number
inp x = number
mul z (2) * 3 (3) = 6
eql z (6) == x (6) = 1
normal ending
self.w=0 self.x=6 self.y=0 self.z=1


In [71]:

four_bit = """inp w
add z w
mod z 2
div w 2
add y w
mod y 2
div w 2
add x w
mod x 2
div w 2
mod w 2"""
monad = Cruncher(four_bit, debug=True)
monad.crunch([7])


Crunching: [7]
inp w = 7
add z (0) + w (7) = 7

mod z (7) % 2 (2) = 1

div w (7) // 2 (2) = 3
add y (0) + w (3) = 3
mod y (3) % 2 (2) = 1
div w (3) // 2 (2) = 1
add x (0) + w (1) = 1
mod x (1) % 2 (2) = 1
div w (1) // 2 (2) = 0
mod w (0) % 2 (2) = 0
normal ending
self.w=0 self.x=1 self.y=1 self.z=1


In [288]:
#This needs to be automated

data = open('d24.input').read()
monad = Cruncher(data, debug=False, letter=False, killstep=253)
# 13 6
# 12 3
# 9  3
# 7  2
# 6  7

# 12 = 11 + 2

#                      ?  xx?xxxxxx
monad.crunch(get_list(5_1_9_8_3_9_9_9_9_4_7_9_9_9))
#                     1 2 3 4 5 6 7 8 9 A B C D E
monad.print_regs()

#1E = +6 -2
#BC = +4 -2
#AD = +14 -9
#89 = +3 -3
#27 = +14 -6
#34 = +13 -14
#56 = +6  -0


monad.crunch(get_list(1_1_2_1_1_7_9_1_1_1_1_3_6_5))
#                     1 2 3 4 5 6 7 8 9 A B C D E

monad.print_regs()

w=9 x=0 y=0 z=0
w=5 x=0 y=0 z=0


In [308]:
#Extract data
# grep -Pzo '(?sm)div z 1.*?add y (?!25)\K-?\d+\s(?=mul y x)' d24.input 
from collections import deque

pushes_raw = """6
14
13
6
3
14
4"""



# grep -Pzo '(?m)div z 26\sadd x \K-?\d+\s' d24.input 

pops_raw = """-14
0
-6
-3
-2
-9
-2"""


# grep -Pzo '(?m)div z (?:1|26)\s' d24.input
pushpops = """div z 1
div z 1
div z 1
div z 26
div z 1
div z 26
div z 26
div z 1
div z 26
div z 1
div z 1
div z 26
div z 26
div z 26"""




def find_answer(counter):
    stack = []
    results = {}
    pushes = deque(int(p) for p in pushes_raw.splitlines())
    pops = deque(int(p.strip()) for p in pops_raw.splitlines())
    for i, pp in enumerate(pushpops.splitlines(), start=1):
        if "z 1" in pp:
            stack.append((i, pushes.popleft()))
        elif "z 26" in pp:
            pushi, pushed = stack.pop()
            compare = pops.popleft()
            for a in counter:
                result = a + pushed + compare
                if result < 10 and result > 0:
                    results[pushi] = a
                    results[i] = result
                    break
    return int(''.join( str(v) for k,v in sorted(results.items(), key = lambda kv: kv[0])))



countdown = range(9,0,-1)
countup = range(1,10)

print(find_answer(countdown))
print(find_answer(countup))

51983999947999
11211791111365
